In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./custom.css", "r").read()
    return HTML(styles)
css_styling()

In [ ]:
# For better readability later on, we do some imports and definitions here
import acoular as ac
from pathlib import Path

from traits.api import List, Int, Property

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio

Path('./cache/02').mkdir(parents=True, exist_ok=True)
ac.config.cache_dir = './cache/02'

# initialize a random generator for path deviations
rng = np.random.default_rng(seed = 23)

def play_signal(signal, fs = 44100):
    # Play a signal sound
    display(Audio(signal, rate = fs))
    

def show_signal(signal, play_sound = True, fs = 44100):
    # Display and play a signal sound
    plt.figure(1,(10,3))
    plt.psd(signal, 
            Fs = fs,
            NFFT = 4096)
    plt.show()

    if play_sound:
        play_signal(signal, fs=fs)
    
def plot_setup(traj, mics = False):
    plt.figure(2,(10,3))
    
    if mics:
        # plot observer
        plt.plot(mics.pos[0,:], mics.pos[1,:], 'rx', label = 'observer')

    # plot trajectory
    tmax = max(list(traj.points.keys()))
    times = np.linspace(0, tmax, 100)
    xt, yt, zt = traj.location(times)
    plt.plot(xt, yt, label = 'trajectory')

    # plot the predefined waypoints
    xwp, ywp, zwp = zip(*traj.points.values())
    plt.plot(xwp, ywp, '>', label = 'traj. waypoints')
    for time in traj.points.keys():
        xwp, ywp, zwp = traj.points[time]
        plt.text(xwp+0.5, ywp-3, f'{time:.0f} s', fontsize=7)

    plt.xlabel('$x$ / m')
    plt.ylabel('$y$ / m')
    plt.legend()
    plt.axis('equal')
    plt.show()

    
def show_result(data, caching=True):
    #if caching:
    #    cached_data = ac.Cache(source=data)
    #else:
    cached_data = data
    signal = ac.tools.return_result(cached_data)
    plt.figure(3,(10,5))
    plt.specgram(signal[:,0], 
                 Fs = f_sample,
                 noverlap = 4096-256,
                 NFFT = 4096,
                 vmin=-100,
                 vmax=-50)
    plt.ylim(0,5000)
    plt.colorbar()
    plt.xlabel('$t$ / s')
    plt.ylabel('$f$ / Hz')
    plt.show()
    
    # Play the sound
    display(Audio(signal.T, rate = f_sample))



class DroneSignalGenerator( ac.NoiseGenerator ):
    """
    Class for generating a synthetic multicopter drone signal. 
    This is just a basic example class for demonstration purposes 
    with only few settable and some arbitrary fixed parameters.
    It is not intended to create perfectly realistic signals.
    """

    # List with rotor speeds (for each rotor independently)
    # Default: 1 rotor, 15000 rpm
    rpm_list = List([15000,])

    # Number of blades per rotor
    # Default: 2
    num_blades_per_rotor = Int(2)
    
    # internal identifier
    digest = Property(depends_on=['rpm_list', 'num_blades_per_rotor', 
                                  'rms', 'seed',
                                  'sample_freq', 'num_samples'])

    def _get_digest(self):
        return ac.internal.digest(self)

    def signal( self ):
        """
        function that returns the full signal
        """
        # initialize a random generator for noise generation
        rng = np.random.default_rng(seed = self.seed)
        # use 1/f² broadband noise as basis for the signal
        wn = rng.standard_normal(self.num_samples) # normal distributed values
        wnf = np.fft.rfft(wn) # transform to freq domain
        wnf /= (np.linspace(0.1,1,len(wnf))*5)**2 # spectrum ~ 1/f²
        sig = np.fft.irfft(wnf) # transform to time domain

        # vector with all time instances
        t = np.arange(self.num_samples, dtype=float) / self.sample_freq

        # iterate over all rotors
        for rpm in self.rpm_list:
            f_base = rpm / 60 # rotor speed in Hz

            # randomly set phase of rotor
            phase = rng.uniform() * 2*np.pi
            
            # calculate higher harmonics up to 50 times the rotor speed
            for n in np.arange(50)+1:
                # if we're looking at a blade passing frequency, make it louder
                if n % self.num_blades_per_rotor == 0:
                    amp = 1
                else:
                    amp = 0.2

                # exponentially decrease amplitude for higher freqs with arbitrary factor
                amp *= np.exp(-n/10)
                
                # add harmonic signal component to existing signal
                sig += amp * np.sin(2*np.pi*n * f_base * t + phase) 

        # return signal normalized to given RMS value
        return sig * self.rms / np.std(sig)

<img src="img/DAGA_logo.png" alt="DAGA Logo" style="width:160px; position:absolute; top:20px; right:20px;">
<img src="img/TU-lang.png" alt="TU Logo" style="width:230px; position:absolute; top:0px; right:180px;">

<h1 style="margin-top: 130px; margin-bottom: 50px; color: #A81D1E;">Acoular Workshop: Generating Synthetic Sound Pressure Time Datasets of Multicopter Drone Fly-bys</h1>
<h3 style="margin-top: 50px; margin-bottom: 70px; color: #434343;">Gert Herold, Mikolaj Czuchaj, Adam Kujawski, Oliver Lylloff, Art J. R. Pelling, Ennes Sarradj</h2>


<div style="width: 20px; font-size: 10px; float: right; text-align:right" > 
    2
</div>

## Looking at a real signal – Measurement of a quadcopter fly-by (setup)

![meas_setup](img/sketch_flyby.png)

<div style="width: 20px; font-size: 10px; float: right; text-align:right" > 
    3
</div>

## Looking at a real signal – Measurement of a quadcopter fly-by (result)

![measured_spectrogram](img/spectro_23-04-04_13-12-03_874736_8192_15sg.png)
<audio src="wav/measurement_drone.wav" controls>Browser does not support audio format.</audio>

Important characteristics:
  - tonal components (e.g. rotor bpf + higher harmonics)
  - Doppler effect
  - broadband components
  - interference patterns due to ground reflections

<div style="width: 20px; font-size: 10px; float: right; text-align:right" > 
    4
</div>

## Simulation – Drone as a moving dipole
    
What we need:
- Signal characteristics
- Flight path
- Observer properties
- Medium properties
- Ground reflections

In [ ]:
# The source itself
flying_drone = ac.MovingPointSourceDipole(conv_amp = True)
flying_drone

<div style="width: 20px; font-size: 10px; float: right; text-align:right" > 
    5
</div>

## Signal characteristics

- simple sine signal (for now)

In [ ]:
# Set up a simple tonal signal
f_sample = 44100 # Hz, sampling frequency
t_meas = 10.5 # s, length of signal
sine_signal = ac.SineGenerator(freq = 700,
                               sample_freq = f_sample,
                               num_samples = f_sample * t_meas)

# Add the signal to the the source
flying_drone.signal = sine_signal

play_signal(sine_signal.signal())

<div style="width: 20px; font-size: 10px; float: right; text-align:right" > 
    6
</div>

## Flight path

 - simple straight line (for now) 

In [ ]:
# Define an Acoular Trajectory object
flight_path = ac.Trajectory()

# Simple case: two points at two time instants
flight_path.points = {
    0  : (-90, 6, 10), # at 0 seconds be at (x,y,z) = (-90, 6, 10) meter
    10 : ( 90, 6, 10)  # after 11 seconds, x has changed by 180 m (=> fly-by)
}


# Add the flight path
flying_drone.trajectory = flight_path

plot_setup(flight_path)

<div style="width: 20px; font-size: 10px; float: right; text-align:right" > 
    7
</div>

## Observer & medium properties

 - "array" with two microphones
 - air at standard conditions 

In [ ]:
# Define the array geometry
two_mics = ac.MicGeom()
two_mics.pos_total = np.array([[-0.07, 0.07], # x positions, all values in m
                               [-0.03, 0.03], # y
                               [ 1.7 , 1.7]]) # z
    
# Medium properties
air = ac.Environment(c = 343.)

# Add the flight path and the medium properties
flying_drone.mics = two_mics
flying_drone.env = air

# starts observation 0.3 seconds after signal starts at drone                           
flying_drone.start = 0.3 # s

plot_setup(flight_path, two_mics)

<div style="width: 20px; font-size: 10px; float: right; text-align:right" > 
    8
</div>

## Export & show results


In [ ]:
# Write data stream onto disk for later re-use. 
# This step is not necessary if only needed once or runtime isn't an issue.
flying_drone_cached = ac.Cache(source = flying_drone)

# Prepare wav output.
# If you don't need caching, you can directly put "source = flying_drone" here.
output = ac.WriteWAV(file = 'flying_sine.wav',
                     source = flying_drone_cached, 
                     channels = [0,1]) # export both channels as stereo
# Start the actual export
output.save()

# Plot the signal
show_result(flying_drone_cached)

<div style="width: 20px; font-size: 10px; float: right; text-align:right" > 
    9
</div>

## Change some properties

![flowchart](img/sim_flowchart.png)


In [ ]:
# change the signal to white noise

noise_signal = ac.WNoiseGenerator(sample_freq = f_sample,
                                  num_samples = f_sample * t_meas)

# Add the signal to the the source
flying_drone.signal = noise_signal

# Export wave file
output.file = "flying_wnoise.wav"
output.save()

show_result(flying_drone_cached)

<div style="width: 20px; font-size: 10px; float: right; text-align:right" > 
    10
</div>

### Change the signal to something drone-like

In [ ]:
drone_signal = DroneSignalGenerator(rpm_list = [15010, 14962, 13536, 13007], 
                                    num_blades_per_rotor = 2, 
                                    sample_freq = f_sample, 
                                    num_samples = f_sample * t_meas)

flying_drone.signal = drone_signal

show_signal(drone_signal.signal())

<div style="width: 20px; font-size: 10px; float: right; text-align:right" > 
    11
</div>

### Make the trajectory deviate from a straight path

In [ ]:
waypoints = { 
    t : ((t-5.5) * 16,        # vary x according to approx. flight speed 16 m/s
         6 + rng.uniform(-0.4,0.4), # randomly vary y position up to ±0.4 m around 6 m 
         10 + rng.uniform(-0.5,0.5)) # randomly vary z position up to ±0.5 m around 10 m height
    for t in np.arange(12) } # 11 seconds trajectory, one waypoint each second

flight_path.points = waypoints

# Export wave file
output.file = "flying_drone.wav"
output.save()

plot_setup(flight_path, two_mics)
show_result(flying_drone_cached)

<div style="width: 20px; font-size: 10px; float: right; text-align:right" > 
    12
</div>

## Ground reflections

- simple way: add a mirror source

In [ ]:
waypoints_reflection = { 
    time : (x, y, -z) for time, (x, y, z) in waypoints.items() 
}
flight_path_mirror = ac.Trajectory(points = waypoints_reflection)


# Define a mirror source with the mirrored trajectory
mirror_drone = ac.MovingPointSourceDipole(signal = drone_signal,        
                                          trajectory = flight_path_mirror,
                                          conv_amp = True,
                                          mics = two_mics,
                                          start = 0.3,
                                          env = air) 

# Mix the original source and the mirror source
drone_above_ground = ac.SourceMixer( sources = [flying_drone,
                                                mirror_drone] )

flying_drone_cached.source = drone_above_ground

# Export wave file
output.file = "flying_drone_above_ground.wav"
output.save()

show_result(flying_drone_cached)

<div style="width: 20px; font-size: 10px; float: right; text-align:right" > 
    13
</div>

## Summary

![flowchart_mixer](img/sim_flowchart2.png)

 - Acoular for simulating an acoustic scenario
 - Moving source with custom signal and dipole characteristic
 - Ground reflections
 - Simple case with point source and two omnidirectional mics as "ears"
 
#### Possible enhancements

 - Consider dynamic signal characteristics
 - Distributed source rather than point source
 - Frequency-dependent (arbitrary) directivity
 - Atmospheric damping
 - Ground absorption & more
 - Better binaural rendering